In [1]:
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils import data
import argparse

In [2]:
class DigitCNN(nn.Module):
    def __init__(self):
        super(DigitCNN, self).__init__()
        self.convnet = nn.Sequential(OrderedDict(
            [
                ('layer 1', nn.Conv2d(1, 10, 5)),
                # ('layer 1', nn.Conv2d(1, 3, 5)),
                ('relu 1', nn.ReLU()), # 10 * 24 * 24
                ('maxpool 1', nn.MaxPool2d(2, stride=2)), # 3 * 12 * 12
                ('dropout 1', nn.Dropout2d(p=0.2)),
                ('batchnorm 1', nn.BatchNorm2d(10)),
                ('layer 2', nn.Conv2d(10, 20, kernel_size=5)), # 5 * 8 * 8
                # ('layer 2', nn.Conv2d(3, 5, kernel_size=5)),  # 5 * 8 * 8
                ('relu 2', nn.ReLU()),
                ('maxpool 2', nn.MaxPool2d(2, stride=2)), # 5 * 4 * 4
                ('dropout 2', nn.Dropout2d(p=0.2)),
            ]
        ))
        self.fcn = nn.Sequential(OrderedDict(
        [
            ('linear 1', nn.Linear(320, 150)),
            # ('linear 1', nn.Linear(80, 160)),
            ('relu 3', nn.ReLU()),
            ('linear 2', nn.Linear(150, 10)),
        ]))

    def forward(self, x):
        conv = self.convnet(x)
        lin = conv.view(-1, 320)
        return self.fcn(lin)


In [3]:
checkpoint_nets = []
for i in range(5):
    checkpoint = torch.load("best_"+str(i)+"checkpoint.pth.tar")
    net = DigitCNN()
    net.load_state_dict(checkpoint['state_dict'])
    checkpoint_nets.append(net)


In [4]:
optimizer = nn.CrossEntropyLoss()

In [5]:
test_original_data = np.genfromtxt('test.csv', delimiter=',', dtype=np.float32)

In [6]:
test_images = test_original_data[1:, :]

In [7]:
test_images_normalized = (test_images - test_images.mean()) / test_images.std()
test_dataloader = data.DataLoader(torch.FloatTensor(test_images_normalized), batch_size=64)


In [8]:
test_labels = []

In [9]:
for i, image in enumerate(test_dataloader):    
    show_images = image.numpy()
    image = image.view(-1, 1, 28, 28)
    var_image = Variable(image)
    predictions = []
    for net in checkpoint_nets:
        prediction = net(var_image)
        res = torch.max(prediction, 1)[1].data.numpy()
        predictions.append(res)
    predictions = np.array(predictions)
    predictions = np.transpose(predictions)
    for res in predictions:
        max_num, max_cnt = 0, 0
        for i in range(10):
            num_pred_i = np.count_nonzero(res == i)
            if max_cnt < num_pred_i:
                max_num = i
                max_cnt = num_pred_i
#         print(res, max_num)
        test_labels.append(max_num)
        
                
            

In [10]:
import pandas as pd

In [11]:
submission = pd.DataFrame(data=test_labels, index=range(1, len(test_labels)+1))

In [12]:
submission = submission.rename(index=str, columns={0:'Labels'})
submission.index.name = 'ImageId'

In [13]:
submission.to_csv('./submission.csv')